In [39]:
# Analysing the small part of JSON data and converting into tabular format

In [31]:
import pandas as pd
import json
df = pd.read_csv('Test_Case_dataCleaning.csv', nrows=10)
print(df)

                                        BODY_CLEANED
0  {\n  "meta": {\n    "messageType": "VehicleDat...
1  {\n  "meta": {\n    "messageType": "VehicleDat...
2  {\n  "meta": {\n    "messageType": "VehicleDat...
3  {\n  "meta": {\n    "messageType": "VehicleDat...
4  {\n  "meta": {\n    "messageType": "VehicleDat...
5  {\n  "meta": {\n    "messageType": "VehicleDat...
6  {\n  "meta": {\n    "messageType": "VehicleDat...
7  {\n  "meta": {\n    "messageType": "VehicleDat...
8  {\n  "meta": {\n    "messageType": "VehicleDat...
9  {\n  "meta": {\n    "messageType": "VehicleDat...


In [21]:
import pandas as pd
df = pd.read_csv('Test_Case_dataCleaning.csv', nrows=1)
raw_json = df.loc[0, 'BODY_CLEANED']
print(raw_json.encode('utf-8').decode('unicode_escape'))


{
  "meta": {
    "messageType": "VehicleData",
    "pollerFetchTime": 1751725939315,
    "version": "v2"
  },
  "partitionKey": "df979dc9-5cc6-470f-a105-d3108fd03c19",
  "timestamp": "20250705143219.297",
  "transactionId": "df979dc9-5cc6-470f-a105-d3108fd03c19",
  "vinList": [
    {
      "dischargeWarning": {
        "hazardStatus": 0,
        "tailLampStatus": 0
      },
      "evStatusData": [
        {
          "attribute": "ign3Status",
          "dataType": 1,
          "value": "0"
        }
      ],
      "headLampStatus": {
        "headLamp": 0,
        "leftHighLamp": 0,
        "leftLowLamp": 0,
        "rightHighLamp": 0,
        "rightLowLamp": 0
      },
      "locationTimestamp": {
        "offSet": -7,
        "time": "20250705143213.000"
      },
      "statusData": [
        {
          "attribute": "odometer",
          "dataType": 2,
          "unit": 1,
          "value": "28082.6"
        },
        {
          "attribute": "gasLevel",
          "dataType": 1,

In [25]:
import pandas as pd
import json

# Extract the JSON string from the first row
raw_json = df.loc[0, 'BODY_CLEANED']

# Clean the JSON string (if needed)
cleaned_json = raw_json.replace('\\n', '').replace('\\"', '"').replace('\\', '')

# Parse the JSON string
parsed_json = json.loads(cleaned_json)

# Extract meta information
poller_fetch_time = parsed_json["meta"].get("pollerFetchTime", None)

# Get the list of vehicles (usually length 1)
vin_list = parsed_json.get("vinList", [])

# Flatten the structure
parsed_data = []

for vin_data in vin_list:
    vin = vin_data.get("vin", None)
    status_timestamp = vin_data.get("statusTimestamp", {}).get("time", None)
    
    for item in vin_data.get("statusData", []):
        parsed_data.append({
            "VIN": vin,
            "statusTimestamp": status_timestamp,
            "attribute": item.get("attribute", None),
            "value": item.get("value", None),
            "pollerFetchTime": poller_fetch_time
        })

# Convert to DataFrame
flattened_df = pd.DataFrame(parsed_data)

# Display the result
print(flattened_df)


             VIN     statusTimestamp          attribute    value  \
0   MASKEDVIN003  20250705143210.000           odometer  28082.6   
1   MASKEDVIN003  20250705143210.000           gasLevel       24   
2   MASKEDVIN003  20250705143210.000    parkingPosition        1   
3   MASKEDVIN003  20250705143210.000    engineOilStatus        0   
4   MASKEDVIN003  20250705143210.000            airTemp        0   
5   MASKEDVIN003  20250705143210.000     breakOilStatus        0   
6   MASKEDVIN003  20250705143210.000          ignStatus        1   
7   MASKEDVIN003  20250705143210.000      accModeStatus        1   
8   MASKEDVIN003  20250705143210.000          batStatus       92   
9   MASKEDVIN003  20250705143210.000  washerFluidStatus        1   
10  MASKEDVIN003  20250705143210.000    traveldirection      268   
11  MASKEDVIN003  20250705143210.000              speed        0   

    pollerFetchTime  
0     1751725939315  
1     1751725939315  
2     1751725939315  
3     1751725939315  
4    

In [29]:
import pandas as pd
import json

# Step 1: Load and clean the first row of JSON
raw_json = df.loc[0, 'BODY_CLEANED']
cleaned_json = raw_json.replace('\\n', '').replace('\\"', '"').replace('\\', '')
parsed_json = json.loads(cleaned_json)

# Step 2: Extract required info
poller_fetch_time = parsed_json["meta"].get("pollerFetchTime", None)
vin_list = parsed_json.get("vinList", [])

# Step 3: Define the attributes you want to keep
wanted_attributes = {"odometer", "gasLevel", "ignStatus", "speed"}

# Step 4: Build the final row
records = []
for vin_data in vin_list:
    flat_row = {
        "VIN": vin_data.get("vin", None),
        "statusTimestamp": vin_data.get("statusTimestamp", {}).get("time", None),
        "pollerFetchTime": poller_fetch_time
    }
    
    # Pull only the desired attributes
    for item in vin_data.get("statusData", []):
        attr = item.get("attribute", None)
        if attr in wanted_attributes:
            flat_row[attr] = item.get("value", None)
    
    records.append(flat_row)

# Step 5: Convert to DataFrame
final_df = pd.DataFrame(records)

# Reorder columns (optional)
final_df = final_df[["VIN", "statusTimestamp", "odometer", "gasLevel", "ignStatus", "speed", "pollerFetchTime"]]

# Display result
print(final_df.to_string(index=False))


         VIN    statusTimestamp odometer gasLevel ignStatus speed  pollerFetchTime
MASKEDVIN003 20250705143210.000  28082.6       24         1     0    1751725939315


In [35]:
import pandas as pd
import json

# Store processed rows
flattened_data = []

# Iterate through all rows in df (already limited to 10 rows)
for index, row in df.iterrows():
    try:
        raw_json = row['BODY_CLEANED'].strip()
        clean_json = raw_json.replace('\\n', '').replace('\\', '').replace('\"', '"')
        data = json.loads(clean_json)

        pollerFetchTime = data.get("meta", {}).get("pollerFetchTime", None)
        vinList = data.get("vinList", [])

        for vin_entry in vinList:
            vin = vin_entry.get("vin", None)
            statusTimestamp = vin_entry.get("statusTimestamp", {}).get("time", None)
            statusData = vin_entry.get("statusData", [])

            # Initialize row
            row_dict = {
                "VIN": vin,
                "statusTimestamp": statusTimestamp,
                "pollerFetchTime": pollerFetchTime,
                "odometer": None,
                "gasLevel": None,
                "ignStatus": None,
                "speed": None
            }

            for item in statusData:
                attr = item.get("attribute")
                value = item.get("value")
                if attr in row_dict:
                    row_dict[attr] = value

            flattened_data.append(row_dict)

    except Exception as e:
        print(f"Error in row {index}: {e}")

# Create the final DataFrame
final_df = pd.DataFrame(flattened_data)

# Show result without slashes and clean format
print(final_df.to_string(index=False))


         VIN    statusTimestamp  pollerFetchTime odometer gasLevel ignStatus speed
MASKEDVIN003 20250705143210.000    1751725939315  28082.6       24         1     0
MASKEDVIN003 20250708170037.000    1751994040672  28605.6      100         1    28
MASKEDVIN003 20250706181414.000    1751825658365  28443.6       37         0     0
MASKEDVIN003 20250708204314.000    1752007396151  28902.2       72         1    55
MASKEDVIN003 20250705202326.000    1751747010652  28275.5       63         1     0
MASKEDVIN003 20250709183810.000    1752086292389  29296.9       23         0     0
MASKEDVIN003 20250709015822.000    1752026304311  29212.4       33         1    34
MASKEDVIN003 20250709182637.000    1752085600580  29296.1       23         1     6
MASKEDVIN003 20250707011609.000    1751850974094  28516.7       25         1     2
MASKEDVIN003 20250707022311.000    1751854993858  28533.7       22         1     3


In [41]:
## Making the full dataset into tabular format

In [257]:
import pandas as pd
import json
df = pd.read_csv('Test_Case_dataCleaning.csv')
print(df.head(10))

                                        BODY_CLEANED
0  {\n  "meta": {\n    "messageType": "VehicleDat...
1  {\n  "meta": {\n    "messageType": "VehicleDat...
2  {\n  "meta": {\n    "messageType": "VehicleDat...
3  {\n  "meta": {\n    "messageType": "VehicleDat...
4  {\n  "meta": {\n    "messageType": "VehicleDat...
5  {\n  "meta": {\n    "messageType": "VehicleDat...
6  {\n  "meta": {\n    "messageType": "VehicleDat...
7  {\n  "meta": {\n    "messageType": "VehicleDat...
8  {\n  "meta": {\n    "messageType": "VehicleDat...
9  {\n  "meta": {\n    "messageType": "VehicleDat...


In [45]:
import pandas as pd
import json

# Store processed rows
flattened_data = []

# Iterate through all rows in df (already limited to 10 rows)
for index, row in df.iterrows():
    try:
        raw_json = row['BODY_CLEANED'].strip()
        clean_json = raw_json.replace('\\n', '').replace('\\', '').replace('\"', '"')
        data = json.loads(clean_json)

        pollerFetchTime = data.get("meta", {}).get("pollerFetchTime", None)
        vinList = data.get("vinList", [])

        for vin_entry in vinList:
            vin = vin_entry.get("vin", None)
            statusTimestamp = vin_entry.get("statusTimestamp", {}).get("time", None)
            statusData = vin_entry.get("statusData", [])

            # Initialize row
            row_dict = {
                "VIN": vin,
                "statusTimestamp": statusTimestamp,
                "pollerFetchTime": pollerFetchTime,
                "odometer": None,
                "gasLevel": None,
                "ignStatus": None,
                "speed": None
            }

            for item in statusData:
                attr = item.get("attribute")
                value = item.get("value")
                if attr in row_dict:
                    row_dict[attr] = value

            flattened_data.append(row_dict)

    except Exception as e:
        print(f"Error in row {index}: {e}")

# Create the final DataFrame
final_df = pd.DataFrame(flattened_data)

# Show result without slashes and clean format
print(final_df.to_string(index=False))

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [47]:
import pandas as pd
import json

def clean_and_parse_json(row):
    try:
        clean_json = row.replace('\\n', '').replace('\\', '')
        return json.loads(clean_json)
    except:
        return None

# Initialize an empty list to store results
final_results = []

chunksize = 1000
for chunk in pd.read_csv('Test_Case_dataCleaning.csv', chunksize=chunksize):
    chunk['parsed'] = chunk['BODY_CLEANED'].apply(clean_and_parse_json)
    chunk = chunk.dropna(subset=['parsed'])

    extracted_rows = []
    for item in chunk['parsed']:
        for vin_data in item.get('vinList', []):
            extracted_rows.append({
                'VIN': vin_data.get('vin'),
                'statusTimestamp': vin_data.get('statusTimestamp', {}).get('time'),
                'pollerFetchTime': item.get('meta', {}).get('pollerFetchTime'),
                'statusData': vin_data.get('statusData', [])
            })

    # Convert to DataFrame
    vin_df = pd.DataFrame(extracted_rows)

    if vin_df.empty:
        continue

    # Explode and normalize statusData
    status_df = pd.json_normalize(vin_df.explode('statusData')['statusData'])

    # Merge metadata with flattened data
    merged_df = vin_df.drop(columns='statusData').join(status_df)

    # Filter only desired attributes
    filtered = merged_df[merged_df['attribute'].isin(['odometer', 'gasLevel', 'ignStatus', 'speed'])]

    # Pivot to make attributes columns
    pivoted = filtered.pivot_table(index=['VIN', 'statusTimestamp', 'pollerFetchTime'],
                                   columns='attribute',
                                   values='value',
                                   aggfunc='first').reset_index()

    final_results.append(pivoted)

# Combine all results
final_df = pd.concat(final_results, ignore_index=True)

# Optional: Reorder columns
final_df = final_df[['VIN', 'statusTimestamp', 'odometer', 'gasLevel', 'ignStatus', 'speed', 'pollerFetchTime']]

# Save to CSV if needed
final_df.to_csv('flattened_output.csv', index=False)


In [49]:
import pandas as pd

# Read the saved file
df = pd.read_csv('flattened_output.csv')

# Display the first few rows
df.head()


,VIN,statusTimestamp,odometer,gasLevel,ignStatus,speed,pollerFetchTime
0,MASKEDVIN003,2.025070e+13,NaN,NaN,NaN,0.0,1751381150911
1,MASKEDVIN003,2.025070e+13,28431.2,NaN,NaN,NaN,1751381630994
2,MASKEDVIN003,2.025070e+13,NaN,33.0,NaN,NaN,1751382014325
3,MASKEDVIN003,2.025070e+13,NaN,99.0,NaN,NaN,1751382247200
4,MASKEDVIN003,2.025070e+13,NaN,NaN,NaN,68.0,1751382784292


In [51]:
len(df)

261341

In [55]:
df = pd.read_csv('Test_Case_dataCleaning.csv')

In [259]:
import pandas as pd
import json

# Target attributes to extract
target_attributes = ['odometer', 'gasLevel', 'ignStatus', 'speed']

# Step 3: Create a list to collect processed rows
processed_data = []

# Step 4: Iterate through each row and parse the JSON
for raw_row in df['BODY_CLEANED']:
    try:
        # Clean and parse JSON string
        cleaned_row = raw_row.replace('\\n', '').replace('\\', '')
        data = json.loads(cleaned_row)

        for vin_info in data.get('vinList', []):  # Process all VINs, not just the first
            row_data = {
                'VIN': vin_info.get('vin'),
                'statusTimestamp': vin_info.get('statusTimestamp', {}).get('time'),
                'pollerFetchTime': data.get('meta', {}).get('pollerFetchTime')
            }

            # Extract only target attributes
            for attr in vin_info.get('statusData', []):
                key = attr.get('attribute')
                if key in target_attributes:
                    row_data[key] = attr.get('value')

            processed_data.append(row_data)

    except (json.JSONDecodeError, IndexError, TypeError) as e:
        print(f"Skipping row due to error: {e}")
        continue

# Step 5: Convert to DataFrame
flattened_df = pd.DataFrame(processed_data)

# Step 6: Prevent scientific notation and keep timestamps as string
flattened_df['statusTimestamp'] = pd.to_numeric(flattened_df['statusTimestamp'], errors='coerce').astype('Int64').astype(str)
flattened_df['pollerFetchTime'] = pd.to_numeric(flattened_df['pollerFetchTime'], errors='coerce').astype('Int64').astype(str)

# Step 7: Save to CSV
flattened_df.to_csv('flattened_vehicle_data.csv', index=False)

# Display the result
print(flattened_df.head())


            VIN statusTimestamp pollerFetchTime odometer gasLevel ignStatus  \
0  MASKEDVIN003  20250705143210   1751725939315  28082.6       24         1   
1  MASKEDVIN003  20250708170037   1751994040672  28605.6      100         1   
2  MASKEDVIN003  20250706181414   1751825658365  28443.6       37         0   
3  MASKEDVIN003  20250708204314   1752007396151  28902.2       72         1   
4  MASKEDVIN003  20250705202326   1751747010652  28275.5       63         1   

  speed  
0     0  
1    28  
2     0  
3    55  
4     0  


In [59]:
len(df)

781942

In [61]:
# Method 1: Manually parses and cleans each row with nested loops — slow and not scalable.

# Method 2: Uses chunking and pivoting with explode — misses data and overcomplicates processing.

# Method 3: Simply parses rows and extracts needed fields — accurate, complete, and most efficient.

In [261]:
import pandas as pd

# Read the saved file
df = pd.read_csv('flattened_vehicle_data.csv')

# Display the first few rows
df.head()

,VIN,statusTimestamp,pollerFetchTime,odometer,gasLevel,ignStatus,speed
0,MASKEDVIN003,2.025071e+13,1751725939315,28082.6,24.0,1.0,0.0
1,MASKEDVIN003,2.025071e+13,1751994040672,28605.6,100.0,1.0,28.0
2,MASKEDVIN003,2.025071e+13,1751825658365,28443.6,37.0,0.0,0.0
3,MASKEDVIN003,2.025071e+13,1752007396151,28902.2,72.0,1.0,55.0
4,MASKEDVIN003,2.025071e+13,1751747010652,28275.5,63.0,1.0,0.0


In [189]:
len(df)

781942

In [191]:
# converted the data into tabluar format 

In [195]:
df.tail()

,VIN,statusTimestamp,pollerFetchTime,odometer,gasLevel,ignStatus,speed
781937,MASKEDVIN009,2.025071e+13,1751755006405,13316.1,33.0,1.0,0.0
781938,MASKEDVIN009,2.025070e+13,1751482987547,12202.5,2.0,1.0,32.0
781939,MASKEDVIN009,2.025070e+13,1751419214435,12008.3,47.0,1.0,0.0
781940,MASKEDVIN009,2.025070e+13,1751566953302,12507.9,5.0,1.0,51.0
781941,MASKEDVIN009,2.025071e+13,1752344640711,15634.8,47.0,1.0,35.0


In [197]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [199]:
## Task 1

In [255]:
df['statusTimestamp'] = df['statusTimestamp'].astype('int64')

# 2. Optional: Convert to string (if you want to view it as a raw number)
df['statusTimestamp_str'] = df['statusTimestamp'].astype(str)

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [249]:
df['statusTimestamp']= pd.to_datetime(
    df['statusTimestamp'],
    format="%Y%m%d%H%M%S.%f",
    errors='coerce'
)

In [251]:
print(df.head())

            VIN statusTimestamp  pollerFetchTime  odometer  gasLevel  \
0  MASKEDVIN003             NaT    1751725939315   28082.6      24.0   
1  MASKEDVIN003             NaT    1751994040672   28605.6     100.0   
2  MASKEDVIN003             NaT    1751825658365   28443.6      37.0   
3  MASKEDVIN003             NaT    1752007396151   28902.2      72.0   
4  MASKEDVIN003             NaT    1751747010652   28275.5      63.0   

   ignStatus  speed  
0        1.0    0.0  
1        1.0   28.0  
2        0.0    0.0  
3        1.0   55.0  
4        1.0    0.0  


In [205]:
df['pollerFetchTime'] = pd.to_datetime(df['pollerFetchTime'], unit='ns', errors='coerce')

In [207]:
print(df.head())

            VIN               statusTimestamp               pollerFetchTime  \
0  MASKEDVIN003 1970-01-01 05:37:30.705143210 1970-01-01 00:29:11.725939315   
1  MASKEDVIN003 1970-01-01 05:37:30.708170037 1970-01-01 00:29:11.994040672   
2  MASKEDVIN003 1970-01-01 05:37:30.706181414 1970-01-01 00:29:11.825658365   
3  MASKEDVIN003 1970-01-01 05:37:30.708204314 1970-01-01 00:29:12.007396151   
4  MASKEDVIN003 1970-01-01 05:37:30.705202326 1970-01-01 00:29:11.747010652   

   odometer  gasLevel  ignStatus  speed  
0   28082.6      24.0        1.0    0.0  
1   28605.6     100.0        1.0   28.0  
2   28443.6      37.0        0.0    0.0  
3   28902.2      72.0        1.0   55.0  
4   28275.5      63.0        1.0    0.0  


In [209]:
df.sort_values('pollerFetchTime', inplace=True)

In [211]:
max_status_time = pd.Timestamp.min
mask = []
for ts in df['statusTimestamp']:
    if ts >= max_status_time:
        mask.append(True)
        max_status_time = ts
    else:
        mask.append(False)

In [212]:
cleaned_df = df[mask].reset_index(drop=True)

In [215]:
len(cleaned_df)

460357

In [217]:
print(cleaned_df)

                 VIN               statusTimestamp  \
0       MASKEDVIN002 1970-01-01 05:37:30.701140112   
1       MASKEDVIN002 1970-01-01 05:37:30.701140112   
2       MASKEDVIN002 1970-01-01 05:37:30.701140112   
3       MASKEDVIN002 1970-01-01 05:37:30.701140402   
4       MASKEDVIN012 1970-01-01 05:37:30.701141440   
...              ...                           ...   
460352  MASKEDVIN012 1970-01-01 05:37:30.716030403   
460353  MASKEDVIN012 1970-01-01 05:37:30.716030412   
460354  MASKEDVIN012 1970-01-01 05:37:30.716030423   
460355  MASKEDVIN012 1970-01-01 05:37:30.716030429   
460356  MASKEDVIN012 1970-01-01 05:37:30.716030432   

                     pollerFetchTime  odometer  gasLevel  ignStatus  speed  
0      1970-01-01 00:29:11.378474562   14634.9      19.0        0.0    0.0  
1      1970-01-01 00:29:11.378481575   14634.9      19.0        0.0    0.0  
2      1970-01-01 00:29:11.378643552   14634.9      19.0        0.0    0.0  
3      1970-01-01 00:29:11.378644554   1463

In [ ]:
## Task 2

In [179]:
cleaned_df.sort_values(by=['statusTimestamp', 'pollerFetchTime'], inplace=True)

In [219]:
cleaned_df = cleaned_df.drop_duplicates(subset=['VIN', 'statusTimestamp'], keep='first')

In [221]:
cleaned_df.reset_index(drop=True, inplace=True)

In [223]:
print(cleaned_df)

                 VIN               statusTimestamp  \
0       MASKEDVIN002 1970-01-01 05:37:30.701140112   
1       MASKEDVIN002 1970-01-01 05:37:30.701140402   
2       MASKEDVIN012 1970-01-01 05:37:30.701141440   
3       MASKEDVIN012 1970-01-01 05:37:30.701141730   
4       MASKEDVIN002 1970-01-01 05:37:30.701142617   
...              ...                           ...   
368577  MASKEDVIN012 1970-01-01 05:37:30.716030403   
368578  MASKEDVIN012 1970-01-01 05:37:30.716030412   
368579  MASKEDVIN012 1970-01-01 05:37:30.716030423   
368580  MASKEDVIN012 1970-01-01 05:37:30.716030429   
368581  MASKEDVIN012 1970-01-01 05:37:30.716030432   

                     pollerFetchTime  odometer  gasLevel  ignStatus  speed  
0      1970-01-01 00:29:11.378474562   14634.9      19.0        0.0    0.0  
1      1970-01-01 00:29:11.378644554   14634.9      19.0        0.0    0.0  
2      1970-01-01 00:29:11.379281772   19939.1      59.0        0.0    0.0  
3      1970-01-01 00:29:11.379459673   1993

In [225]:
len(cleaned_df)

368582

In [227]:
## task 3

In [229]:
cleaned_df.sort_values(by=['VIN', 'statusTimestamp'], inplace=True)

In [231]:
cleaned_df['odometer_diff'] = df.groupby('VIN')['odometer'].diff()

In [233]:
cleaned_df = cleaned_df[(cleaned_df['odometer_diff'] >= 0)]

In [235]:
cleaned_df.drop(columns=['odometer_diff'], inplace=True)

In [237]:
cleaned_df.reset_index(drop=True, inplace=True)

In [239]:
print(cleaned_df)

                 VIN               statusTimestamp  \
0       MASKEDVIN001 1970-01-01 05:37:30.701161305   
1       MASKEDVIN001 1970-01-01 05:37:30.701161306   
2       MASKEDVIN001 1970-01-01 05:37:30.701161308   
3       MASKEDVIN001 1970-01-01 05:37:30.701161428   
4       MASKEDVIN001 1970-01-01 05:37:30.701161528   
...              ...                           ...   
336980  MASKEDVIN013 1970-01-01 05:37:30.715235824   
336981  MASKEDVIN013 1970-01-01 05:37:30.715235855   
336982  MASKEDVIN013 1970-01-01 05:37:30.715235925   
336983  MASKEDVIN013 1970-01-01 05:37:30.715235940   
336984  MASKEDVIN013 1970-01-01 05:37:30.715235955   

                     pollerFetchTime      odometer  gasLevel  ignStatus  speed  
0      1970-01-01 00:29:11.386387726  21669.700000      73.0        1.0    NaN  
1      1970-01-01 00:29:11.386388837  21669.700000      73.0        1.0    NaN  
2      1970-01-01 00:29:11.386390711  21669.700000      73.0        1.0    NaN  
3      1970-01-01 00:29:11.

In [241]:
len(cleaned_df)

336985